In [1]:
!pip install -qqq git+https://github.com/chalk-diagrams/planar git+https://github.com/danoneata/chalk@srush-patch-1
!wget -q https://github.com/srush/GPU-Puzzles/raw/main/robot.png https://github.com/srush/GPU-Puzzles/raw/main/lib.py
!sed -i '448i\            return' lib.py

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 6.5 MB/s eta 0:00:00


In [2]:
import numba
import numpy as np
import warnings
from lib import CudaProblem, Coord
warnings.filterwarnings(
    action="ignore", category=numba.NumbaPerformanceWarning, module="numba"
)

In [3]:
# P1
def map_spec(a):
    return a + 10


def map_test(cuda):
    def call(out, a) -> None:
        local_i = cuda.threadIdx.x
        # FILL ME IN (roughly 1 lines)
        out[local_i] = a[local_i] + 10

    return call


SIZE = 4
out = np.zeros((SIZE,))
a = np.arange(SIZE)
problem = CudaProblem(
    "Map", map_test, [a], out, threadsperblock=Coord(SIZE, 1), spec=map_spec
)
# problem.show()
problem.check()

Passed Tests!


In [4]:
# P2
def zip_spec(a, b):
    return a + b


def zip_test(cuda):
    def call(out, a, b) -> None:
        local_i = cuda.threadIdx.x
        # FILL ME IN (roughly 1 lines)
        out[local_i] = a[local_i] + b[local_i]

    return call


SIZE = 4
out = np.zeros((SIZE,))
a = np.arange(SIZE)
b = np.arange(SIZE)
problem = CudaProblem(
    "Zip", zip_test, [a, b], out, threadsperblock=Coord(SIZE, 1), spec=zip_spec
)
# problem.show()
problem.check()

Passed Tests!


In [5]:
# P3
def map_guard_test(cuda):
    def call(out, a, size) -> None:
        local_i = cuda.threadIdx.x
        # FILL ME IN (roughly 2 lines)
        if local_i < size:
            out[local_i] = a[local_i] + 10

    return call


SIZE = 4
out = np.zeros((SIZE,))
a = np.arange(SIZE)
problem = CudaProblem(
    "Guard",
    map_guard_test,
    [a],
    out,
    [SIZE],
    threadsperblock=Coord(8, 1),
    spec=map_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [6]:
# P4
def map_2D_test(cuda):
    def call(out, a, size) -> None:
        local_i = cuda.threadIdx.x
        local_j = cuda.threadIdx.y
        # FILL ME IN (roughly 2 lines)
        if local_i < size and local_j < size:
            out[local_i, local_j] = a[local_i, local_j] + 10

    return call


SIZE = 2
out = np.zeros((SIZE, SIZE))
a = np.arange(SIZE * SIZE).reshape((SIZE, SIZE))
problem = CudaProblem(
    "Map 2D", map_2D_test, [a], out, [SIZE], threadsperblock=Coord(3, 3), spec=map_spec
)
# problem.show()
problem.check()

Passed Tests!


In [7]:
# P5
def broadcast_test(cuda):
    def call(out, a, b, size) -> None:
        local_i = cuda.threadIdx.x
        local_j = cuda.threadIdx.y
        # FILL ME IN (roughly 2 lines)
        if local_i < size and local_j < size:
            out[local_i, local_j] = a[local_i, 0] + b[0, local_j]

    return call


SIZE = 2
out = np.zeros((SIZE, SIZE))
a = np.arange(SIZE).reshape(SIZE, 1)
b = np.arange(SIZE).reshape(1, SIZE)
problem = CudaProblem(
    "Broadcast",
    broadcast_test,
    [a, b],
    out,
    [SIZE],
    threadsperblock=Coord(3, 3),
    spec=zip_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [8]:
# P6
def map_block_test(cuda):
    def call(out, a, size) -> None:
        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        # FILL ME IN (roughly 2 lines)
        if i < size:
            out[i] = a[i] + 10

    return call


SIZE = 9
out = np.zeros((SIZE,))
a = np.arange(SIZE)
problem = CudaProblem(
    "Blocks",
    map_block_test,
    [a],
    out,
    [SIZE],
    threadsperblock=Coord(4, 1),
    blockspergrid=Coord(3, 1),
    spec=map_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [9]:
# P7
def map_block2D_test(cuda):
    def call(out, a, size) -> None:
        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        # FILL ME IN (roughly 4 lines)
        j = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
        if i < size and j < size:
            out[i, j] = a[i, j] + 10

    return call


SIZE = 5
out = np.zeros((SIZE, SIZE))
a = np.ones((SIZE, SIZE))

problem = CudaProblem(
    "Blocks 2D",
    map_block2D_test,
    [a],
    out,
    [SIZE],
    threadsperblock=Coord(3, 3),
    blockspergrid=Coord(2, 2),
    spec=map_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [10]:
# P8
TPB = 4
def shared_test(cuda):
    def call(out, a, size) -> None:
        shared = cuda.shared.array(TPB, numba.float32)
        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        local_i = cuda.threadIdx.x

        if i < size:
            shared[local_i] = a[i]

        # FILL ME IN (roughly 2 lines)
        cuda.syncthreads()
        if i < size:
            out[i] = shared[local_i] + 10


    return call


SIZE = 8
out = np.zeros(SIZE)
a = np.ones(SIZE)
problem = CudaProblem(
    "Shared",
    shared_test,
    [a],
    out,
    [SIZE],
    threadsperblock=Coord(TPB, 1),
    blockspergrid=Coord(2, 1),
    spec=map_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [11]:
# P9
def pool_spec(a):
    out = np.zeros(*a.shape)
    for i in range(a.shape[0]):
        out[i] = a[max(i - 2, 0) : i + 1].sum()
    return out


TPB = 8
def pool_test(cuda):
    def call(out, a, size) -> None:
        shared = cuda.shared.array(TPB, numba.float32)
        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        local_i = cuda.threadIdx.x
        # FILL ME IN (roughly 8 lines)
        if i < size:
            shared[local_i] = a[i]
        cuda.syncthreads()
        if i < size:
            sum = shared[local_i]
            for t in range(1, 3):
                if local_i - t >= 0:
                    sum = sum + shared[local_i - t]
                elif i - t >= 0:
                    sum = sum + a[i - t]
            out[i] = sum

    return call


SIZE = 8
out = np.zeros(SIZE)
a = np.arange(SIZE)
problem = CudaProblem(
    "Pooling",
    pool_test,
    [a],
    out,
    [SIZE],
    threadsperblock=Coord(TPB, 1),
    blockspergrid=Coord(1, 1),
    spec=pool_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [12]:
# P10
def dot_spec(a, b):
    return a @ b

TPB = 8
def dot_test(cuda):
    def call(out, a, b, size) -> None:
        shared = cuda.shared.array(TPB, numba.float32)

        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        local_i = cuda.threadIdx.x
        # FILL ME IN (roughly 9 lines)
        if i < size:
            a_i = a[i]
            b_i = b[i]
            shared[local_i] = a_i * b_i
        cuda.syncthreads()
        step = 1
        while step < cuda.blockDim.x:
            if i < size and local_i % step == 0 and i + step < size and local_i + step < cuda.blockDim.x:
                shared[local_i] = shared[local_i] + shared[local_i + step]
            cuda.syncthreads()
            step = step * 2
        if local_i == 0:
            # out[0] = shared[local_i]
            numba.cuda.atomic.add(out, 0, shared[local_i])
    return call


SIZE = 155
out = np.zeros(1)
a = np.arange(SIZE)
b = np.arange(SIZE)
problem = CudaProblem(
    "Dot",
    dot_test,
    [a, b],
    out,
    [SIZE],
    threadsperblock=Coord(TPB, 1),
    blockspergrid=Coord((SIZE + TPB - 1) // TPB, 1),
    spec=dot_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [13]:
# P11
def conv_spec(a, b):
    out = np.zeros(*a.shape)
    len = b.shape[0]
    for i in range(a.shape[0]):
        out[i] = sum([a[i + j] * b[j] for j in range(len) if i + j < a.shape[0]])
    return out


MAX_CONV = 4
TPB = 8
TPB_MAX_CONV = TPB + MAX_CONV
def conv_test(cuda):
    def call(out, a, b, a_size, b_size) -> None:
        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        local_i = cuda.threadIdx.x

        # FILL ME IN (roughly 17 lines)
        shared_a = cuda.shared.array(TPB_MAX_CONV, numba.float32)
        shared_b = cuda.shared.array(MAX_CONV, numba.float32)

        if i < a_size:
          shared_a[local_i] = a[i]
        if local_i < b_size - 1 and i + cuda.blockDim.x < a_size:
          shared_a[local_i + cuda.blockDim.x] = a[i + cuda.blockDim.x]
        if local_i < b_size:
          shared_b[local_i] = b[local_i]
        cuda.syncthreads()

        sum = 0
        for j in range(b_size):
          if local_i + j < cuda.blockDim.x+b_size - 1:
            sum = sum + shared_a[local_i + j] * shared_b[j]
        if i < a_size:
          out[i] = sum
    return call


# Test 1

SIZE = 6
CONV = 3
out = np.zeros(SIZE)
a = np.arange(SIZE)
b = np.arange(CONV)
problem = CudaProblem(
    "1D Conv (Simple)",
    conv_test,
    [a, b],
    out,
    [SIZE, CONV],
    Coord(1, 1),
    Coord(TPB, 1),
    spec=conv_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [14]:
# Test 2

SIZE = 125
CONV = 3
out = np.zeros(SIZE)
a = np.arange(SIZE)
b = np.arange(CONV)
problem = CudaProblem(
    "1D Conv (Full)",
    conv_test,
    [a, b],
    out,
    [SIZE, CONV],
    Coord((SIZE+TPB-1) // TPB, 1),
    Coord(TPB, 1),
    spec=conv_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [15]:
# P12
TPB = 8
def sum_spec(a):
    o = np.zeros((a.shape[0] + TPB - 1) // TPB)
    for j, i in enumerate(range(0, a.shape[-1], TPB)):
        o[j] = a[i : i + TPB].sum()
    return o


def sum_test(cuda):
    def call(o, a, size: int) -> None:
        cache = cuda.shared.array(TPB, numba.float32)
        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        local_i = cuda.threadIdx.x
        # FILL ME IN (roughly 12 lines)
        if i < size: # 数据搬运
            cache[local_i] = a[i]
        cuda.syncthreads()

        step = 1

        # 算法第一部分
        while step <= cuda.blockDim.x // 2:
            if i < size and (local_i + 1) % (2 * step) == 0:
                cache[local_i] = cache[local_i] + cache[local_i - step]
            cuda.syncthreads()
            step = step * 2

        # 算法第二部分
        while step >= 1:
            if i < size and (local_i + 1) % (2 * step) == 0 and local_i + step < cuda.blockDim.x:
                cache[local_i + step] = cache[local_i + step] + cache[local_i]
            cuda.syncthreads()
            step = step // 2

        # 除了最后一段，前面每段的末尾
        if i < size - 1 and local_i + 1 == cuda.blockDim.x:
            o[cuda.blockIdx.x] = cache[local_i]
        # 最后一段的末尾
        if i == size - 1:
            o[cuda.blockIdx.x] = cache[local_i]

    return call

In [16]:
# Test 1

SIZE = 8
out = np.zeros(1)
inp = np.arange(SIZE)
problem = CudaProblem(
    "Sum (Simple)",
    sum_test,
    [inp],
    out,
    [SIZE],
    Coord(1, 1),
    Coord(TPB, 1),
    spec=sum_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [17]:
# Test 2

SIZE = 16
BLOCKS = (SIZE + TPB - 1) // TPB
out = np.zeros(BLOCKS)
inp = np.arange(SIZE)
problem = CudaProblem(
    "Sum (Full)",
    sum_test,
    [inp],
    out,
    [SIZE],
    Coord(BLOCKS, 1),
    Coord(TPB, 1),
    spec=sum_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [18]:
# P13
TPB = 8
def sum_spec(a):
    out = np.zeros((a.shape[0], (a.shape[1] + TPB - 1) // TPB))
    for j, i in enumerate(range(0, a.shape[-1], TPB)):
        out[..., j] = a[..., i : i + TPB].sum(-1)
    return out


def axis_sum_test(cuda):
    def call(out, a, size: int) -> None:
        cache = cuda.shared.array(TPB, numba.float32)
        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        local_i = cuda.threadIdx.x
        batch = cuda.blockIdx.y
        # FILL ME IN (roughly 12 lines)
        if i < size:
            cache[local_i] = a[batch, i]
        cuda.syncthreads()
        step = 1
        while step < TPB:
            if i < size and local_i % step == 0 and local_i + step < cuda.blockDim.x and i + step < size:
                cache[local_i] = cache[local_i] + cache[local_i + step]
            cuda.syncthreads()
            step = step * 2
        if local_i == 0:
            out[batch, i] = cache[local_i]
        cuda.syncthreads()

    return call


BATCH = 4
SIZE = 6
BLOCKS = (SIZE + TPB - 1) // TPB
out = np.zeros((BATCH, BLOCKS))
inp = np.arange(BATCH * SIZE).reshape((BATCH, SIZE))
problem = CudaProblem(
    "Axis Sum",
    axis_sum_test,
    [inp],
    out,
    [SIZE],
    Coord(BLOCKS, BATCH),
    Coord(TPB, 1),
    spec=sum_spec,
)
# problem.show()
problem.check()

Passed Tests!


In [19]:
# P14
def matmul_spec(a, b):
    return a @ b


TPB = 3
def mm_oneblock_test(cuda):
    def call(out, a, b, size: int) -> None:
        a_shared = cuda.shared.array((TPB, TPB), numba.float32)
        b_shared = cuda.shared.array((TPB, TPB), numba.float32)

        i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        j = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
        local_i = cuda.threadIdx.x
        local_j = cuda.threadIdx.y
        # FILL ME IN (roughly 14 lines)
        tiles = (size + TPB - 1) // TPB

        sum = 0
        for t in range(tiles):
            a_t = 0
            b_t = 0
            k = t * TPB + local_i
            # 搬运的条件很容易出错！
            if j < size and k < size:
                a_t = a[j, k]
            k = t * TPB + local_j
            if i < size and k < size:
                b_t = b[k, i]
            a_shared[local_j, local_i] = a_t
            b_shared[local_j, local_i] = b_t
            cuda.syncthreads()

            if i < size and j < size:
                for s in range(TPB):
                    sum = sum + a_shared[local_j, s] * b_shared[s, local_i]
            cuda.syncthreads() # 这个同步很容易漏掉！
        if i < size and j < size:
            out[j, i] = sum

    return call

# # Test 1

SIZE = 2
out = np.zeros((SIZE, SIZE))
inp1 = np.arange(SIZE * SIZE).reshape((SIZE, SIZE))
inp2 = np.arange(SIZE * SIZE).reshape((SIZE, SIZE)).T

problem = CudaProblem(
    "Matmul (Simple)",
    mm_oneblock_test,
    [inp1, inp2],
    out,
    [SIZE],
    Coord(1, 1),
    Coord(TPB, TPB),
    spec=matmul_spec,
)
# problem.show(sparse=True)

In [20]:
problem.check()

Passed Tests!


In [21]:
SIZE = 8
out = np.zeros((SIZE, SIZE))
inp1 = np.arange(SIZE * SIZE).reshape((SIZE, SIZE))
inp2 = np.arange(SIZE * SIZE).reshape((SIZE, SIZE)).T

problem = CudaProblem(
    "Matmul (Full)",
    mm_oneblock_test,
    [inp1, inp2],
    out,
    [SIZE],
    Coord(3, 3),
    Coord(TPB, TPB),
    spec=matmul_spec,
)
# problem.show(sparse=True)

In [22]:
problem.check()

Passed Tests!
